# Keyword Deletion

Extract keywords using [Yake!](https://doi.org/10.1016/j.ins.2019.09.013)

Gap keywords in the Cloze passsage. If a keyword appears multiple times, gap all instances of the keyword.

In [1]:
import yake 

In [12]:
language = "en"
max_ngram_size = 1
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 6

extractor = yake.KeywordExtractor(
    lan=language,
    n=max_ngram_size,
    dedupLim=deduplication_threshold,
    dedupFunc=deduplication_algo,
    windowsSize=windowSize,
    top=numOfKeywords,
    features=None,
)

In [13]:
def gap(text):
    kws = extractor.extract_keywords(text)
    for kw, score in kws:
        text = text.replace(kw, "_" * len(kw))
    return text

In [14]:
text = """The Cloze procedure, first introduced by Taylor, is a widely used method for creating reading 
comprehension tests inspired by the Gestalt principle of closure. Though many variations have been 
introduced and studied, the core concept is to mask words in prose and task the subject with providing 
the missing words."""

text = """Embarking on an international assignment, whether for work or study, entails navigating a complex 
landscape of emotional and cultural challenges. Initially marked by intrigue and excitement, expatriates often 
face culture shock and a period of adjustment before embracing their host culture. This journey necessitates 
meticulous preparation akin to other significant life changes, emphasizing the importance of adaptability, 
language proficiency, and cultural understanding. Successful expatriates are those who, rather than succumbing 
to frustration, leverage these experiences to enhance their personal and professional growth. The process of 
acculturation involves various emotional stages, including initial elation, culture shock, and eventual acceptance, 
followed by the challenges of reentry into one's native culture. Despite the potential for early termination of 
assignments due to family or personal issues, careful consideration and preparation can mitigate these risks, 
making international experience a valuable asset both personally and professionally."""

cloze_text = gap(text)
print("Cloze text:")
print(cloze_text)
# print("\nAnswers:")
# answer_dict = {answer: {word: round(prob,2) for word, prob in pred.items()} for answer, pred in zip(answers, preds)}
# pp(answer_dict)

Cloze text:
_________ on an international assignment, whether for ____ or _____, _______ navigating a complex 
landscape of emotional and ________ challenges. Initially marked by intrigue and excitement, expatriates often 
face _______ shock and a period of adjustment before embracing their host _______. This journey necessitates 
meticulous preparation akin to other significant life changes, emphasizing the importance of adaptability, 
language proficiency, and ________ understanding. Successful expatriates are those who, rather than succumbing 
to frustration, leverage these experiences to enhance their personal and professional growth. The process of 
acculturation involves various emotional stages, including initial elation, _______ shock, and eventual acceptance, 
followed by the challenges of reentry into one's native _______. Despite the potential for early termination of 
assignments due to family or personal issues, careful consideration and preparation can mitigate these risk